In [26]:
import os
from tqdm import tqdm

import torch
import torch.nn as nn
import numpy as np
from torch.nn.utils import clip_grad_norm_

In [28]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [29]:
# Hyper-parameters
embed_size = 128
hidden_size = 128
num_layers = 1
num_epochs = 10
num_samples = 1000  # number of words to be sampled
batch_size = 20
seq_length = 30
lr = 5e-3

In [30]:
class Dictionary(object): # Actually, starting with python 3.x version, we don't need to inherit object class
    def __init__(self):
        self.word2idx = {} # key : word, val : idx
        self.idx2word = {} # key : idx, val : word
        self.idx = 0

    def add_word(self, word):
        if not word in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1
    
    def __len__(self):
        return len(self.word2idx)
    
class Corpus(object):
    def __init__(self):
        self.dictionary = Dictionary()

    def get_data(self, path, batch_size = 20):
        # Add words to the dictionary
        with open(path, 'r') as f:
            tokens = 0
            for line in f:
                words = line.split() + ['<eos>'] # ['blah1', 'blah2', ..., '<eos>'] 
                tokens += len(words)
                for word in words:
                    self.dictionary.add_word(word)
        
        # Tokenize the file content
        ids = torch.LongTensor(tokens) # create tensor of tokens initialized with random number // try torch.LongTensor(3)
        token = 0
        with open(path, 'r') as f:
            for line in f:
                words = line.split() + ['<eos>']
                for word in words:
                    ids[token] = self.dictionary.word2idx[word]
                    token += 1
        num_batches = ids.size(0) // batch_size
        ids = ids[:num_batches*batch_size]
        return ids.view(batch_size, -1)

In [31]:
# Load "Penn Treebank" dataset
corpus = Corpus()
ids = corpus.get_data('data/train.txt', batch_size)
vocab_size = len(corpus.dictionary)
num_batches = ids.size(1) // seq_length

In [32]:
# RNN based language model
class RNNLM(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(RNNLM, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, vocab_size)
    
    def forward(self, x, h):
        # Embed word ids to vectors
        x = self.embed(x)

        # Forward propagate LSTM
        out, (h,c) = self.lstm(x, h)

        # Reshape output to (batch_size*sequence_length, hidden_size)
        out = out.reshape(out.size(0)*out.size(1), out.size(2))

        # Decode hidden states of all time steps
        out = self.linear(out)
        return out, (h, c)

In [33]:
model = RNNLM(vocab_size, embed_size, hidden_size, num_layers).to(device)

In [34]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [35]:
# Truncated backpropagation
def detach(states):
    return [state.detach() for state in states]
    # See the train code below

In [36]:
# Train the model
log_interval = 300

for epoch in range(1, num_epochs+1):
    # Set initial hidden and cell states
    states = (torch.zeros(num_layers, batch_size, hidden_size).to(device),
              torch.zeros(num_layers, batch_size, hidden_size).to(device))
    
    for i in range(0, ids.size(1) - seq_length, seq_length):
        # Get mini-batch inputs and targets
        inputs = ids[:, i:i+seq_length].to(device)
        targets = ids[:, (i+1):(i+1)+seq_length].to(device)

        # Forward
        states = detach(states)
        outputs, states = model(inputs, states)
        loss = criterion(outputs, targets.reshape(-1))

        # Backward
        optimizer.zero_grad()
        loss.backward()
        clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        step = (i+1) // seq_length
        if step % log_interval == 0:
            print(f'Epoch [{epoch}/{num_epochs}], Step [{step}/{num_batches}, Loss : {loss.item():.4f}, Perplexity : {np.exp(loss.item()):5.2f}]')

Epoch [1/10], Step [0/1549, Loss : 9.2157, Perplexity : 10053.94]
Epoch [1/10], Step [300/1549, Loss : 5.8653, Perplexity : 352.58]
Epoch [1/10], Step [600/1549, Loss : 5.3524, Perplexity : 211.11]
Epoch [1/10], Step [900/1549, Loss : 5.2562, Perplexity : 191.74]
Epoch [1/10], Step [1200/1549, Loss : 5.3735, Perplexity : 215.62]
Epoch [1/10], Step [1500/1549, Loss : 5.3487, Perplexity : 210.34]
Epoch [2/10], Step [0/1549, Loss : 5.6463, Perplexity : 283.25]
Epoch [2/10], Step [300/1549, Loss : 5.0350, Perplexity : 153.70]
Epoch [2/10], Step [600/1549, Loss : 4.8439, Perplexity : 126.96]
Epoch [2/10], Step [900/1549, Loss : 4.7450, Perplexity : 115.00]
Epoch [2/10], Step [1200/1549, Loss : 4.9996, Perplexity : 148.36]
Epoch [2/10], Step [1500/1549, Loss : 4.9720, Perplexity : 144.31]
Epoch [3/10], Step [0/1549, Loss : 5.1317, Perplexity : 169.30]
Epoch [3/10], Step [300/1549, Loss : 4.6847, Perplexity : 108.28]
Epoch [3/10], Step [600/1549, Loss : 4.6320, Perplexity : 102.72]
Epoch [3/1

In [40]:
# Test the model
with torch.no_grad():
    with open('sample.txt', 'w') as f:
        # Set initial hidden and cell states
        state = (torch.zeros(num_layers, 1, hidden_size).to(device),
                 torch.zeros(num_layers, 1, hidden_size).to(device))
        
        # Select one word id randomly
        prob = torch.ones(vocab_size)
        input = torch.multinomial(prob, num_samples=1).unsqueeze(1).to(device) # return idx

        for i in range(num_samples):
            # Forward
            output, state = model(input, state)

            # Sample a word id
            prob = output.exp()
            word_id = torch.multinomial(prob, num_samples=1).item()
            
            # Fill input with sampled word id for the next time step
            input.fill_(word_id)

            # File write
            word = corpus.dictionary.idx2word[word_id]
            word = '\n' if word == '<eos>' else word + ' '
            f.write(word)

            if (i+1)%100 == 0:
                print(f'Sampled [{i+1}/{num_samples}] words and save to {"sample.txt"}')

Sampled [100/1000] words and save to sample.txt
Sampled [200/1000] words and save to sample.txt
Sampled [300/1000] words and save to sample.txt
Sampled [400/1000] words and save to sample.txt
Sampled [500/1000] words and save to sample.txt
Sampled [600/1000] words and save to sample.txt
Sampled [700/1000] words and save to sample.txt
Sampled [800/1000] words and save to sample.txt
Sampled [900/1000] words and save to sample.txt
Sampled [1000/1000] words and save to sample.txt


In [ ]:
torch.save(model.state_dict(), 'model.ckpt')